In [1]:
import codecs
import cStringIO
import csv
import json
import movie_scrapers
import operator
import sys

# STEP 1: Generate List of Animated Movies Released Since 1980

In [3]:
# Generate list of wiki URLs showing animated movies released each year (1 url per year)
urls = []
BASE_URL = "https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_"
year = 1980
year_list = []
while year < 2017:
    urls.append(BASE_URL+"{}".format(year))
    year_list.append(year)
    year += 1

In [42]:
# Call the function scrape_movie_lists for each year, starting with 1980
# Make a dictionary with year as key mapped to list of movies
movie_titles_all = {}
for idx, url in enumerate(urls):
    movie_list = []    
    movie_list.append(movie_scrapers.scrape_movie_lists(url))
    movie_list = reduce(operator.concat, movie_list)
    movie_titles_all[year_list[idx]] = movie_list

In [21]:
# Copied from python documentation:
class UnicodeWriter:
    """
    A CSV writer which will write rows to CSV file "f",
    which is encoded in the given encoding.
    """

    def __init__(self, f, dialect=csv.excel, encoding="utf-8", **kwds):
        # Redirect output to a queue
        self.queue = cStringIO.StringIO()
        self.writer = csv.writer(self.queue, dialect=dialect, **kwds)
        self.stream = f
        self.encoder = codecs.getincrementalencoder(encoding)()

    def writerow(self, row):
        self.writer.writerow([s.encode("utf-8") for s in row])
        # Fetch UTF-8 output from the queue ...
        data = self.queue.getvalue()
        data = data.decode("utf-8")
        # ... and reencode it into the target encoding
        data = self.encoder.encode(data)
        # write to the target stream
        self.stream.write(data)
        # empty queue
        self.queue.truncate(0)

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [27]:
# Save the list of animated movies in a json file
# release year is dict key, mapped to list of movies released that year
with open('titles_by_year.json','wb') as outfile:
    json.dump(movie_titles_all, outfile, indent = 2, separators = (',', ': '))

In [11]:
with open('titles_by_year.json','rb') as infile:
    movie_dicts = json.load(infile)

# STEP 2: Scrape IMDB for URL Key

In [18]:
# Call the function get_imdb_links for each movie: 
imdb_urls = []
current_idx = 0
m = 2633
for key in movie_dicts.keys():
    list_by_year = movie_dicts[key]
    for movie in list_by_year:
        imdb_urls.append(movie_scrapers.get_imdb_links(movie, key))
    
        if (current_idx % 10) == 0:
            progress = 100.0 * current_idx / m
            sys.stdout.write('\r progress {} / {} ({:6.2f}%)'.format(current_idx, m, progress))
            sys.stdout.flush()
        current_idx += 1
        
print len(imdb_urls)

 progress 2630 / 2633 ( 99.89%)2632


In [22]:
# Save the list of imdb movie links in a csv file:
with open('imdb_url_list.csv', 'wb') as output:
    writer = UnicodeWriter(output, lineterminator='\n', quoting=csv.QUOTE_NONNUMERIC)
    for val in imdb_urls:
        if val:
            writer.writerow([val])  

Note that the length of the CSV file is only 987 rows. I spot checked a few movies, and most of the missing movies are foreign animated movies.